In [1]:
from statsbombpy import sb
import pandas as pd
import numpy as np

In [2]:
comp=sb.competitions()

/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [3]:
match=sb.matches(competition_id=55,season_id=282)

In [4]:
spain_match=match[(match['home_team']=='Spain') | (match['away_team']=='Spain')]

In [5]:
spain_match=spain_match[['match_id','home_team', 'away_team', 'home_score', 'away_score','competition_stage']]

In [6]:
spain_match_ids=list(spain_match['match_id'])

In [7]:
spain_match_ids_dict=spain_match.groupby('competition_stage')['match_id'].apply(list).to_dict()

In [8]:
final_spain=spain_match_ids_dict['Final']

In [9]:
final_event=sb.events(match_id=final_spain[0])

/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [10]:
prior_ids=set(spain_match_ids)-set(final_spain)
prior_ids=list(prior_ids)

In [11]:
prior_match_event = pd.DataFrame() 

for i in prior_ids:
    match_event = sb.events(match_id=i)
    prior_match_event = pd.concat([prior_match_event, match_event], ignore_index=True)

/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [12]:
prior_match_event=prior_match_event[['location','match_id','minute','pass_end_location','pass_outcome','pass_type','possession_team','team','type',]]
prior_match_pass=prior_match_event[prior_match_event['type']=='Pass']

In [13]:
final_event=final_event[['location','match_id','minute','pass_end_location','pass_outcome','pass_type','possession_team','team','type','period']]
final_event_pass=final_event[final_event['type']=='Pass']

In [14]:
prior_match_pass=prior_match_pass[prior_match_pass['possession_team']==prior_match_pass['team']]
prior_match_pass=prior_match_pass[prior_match_pass.team=='Spain']

In [15]:
final_event_pass=final_event_pass[final_event_pass['possession_team']==final_event_pass['team']]
final_event_pass=final_event_pass[final_event_pass.team=='Spain']
final_event_pass_2=final_event_pass[final_event_pass.period==2]
final_event_pass=final_event_pass[final_event_pass.period==1]

In [16]:
# Function to convert lists to tuples, leave other types unchanged
def convert_to_tuple(x):
    if isinstance(x, list):
        return tuple(x)
    return x

prior_match_pass['pass_end_location'] = prior_match_pass['pass_end_location'].apply(convert_to_tuple)
prior_match_pass['location'] = prior_match_pass['location'].apply(convert_to_tuple)

# Extract x, y, endx, endy coordinates
prior_match_pass['x'] = prior_match_pass['location'].apply(lambda loc: loc[0])
prior_match_pass['endx'] = prior_match_pass['pass_end_location'].apply(lambda loc: loc[0])
prior_match_pass['y'] = prior_match_pass['location'].apply(lambda loc: loc[1])
prior_match_pass['endy'] = prior_match_pass['pass_end_location'].apply(lambda loc: loc[1])


In [17]:
final_event_pass['pass_end_location'] = final_event_pass['pass_end_location'].apply(convert_to_tuple)
final_event_pass['location'] = final_event_pass['location'].apply(convert_to_tuple)

# Extract x, y, endx, endy coordinates
final_event_pass['x'] = final_event_pass['location'].apply(lambda loc: loc[0])
final_event_pass['endx'] = final_event_pass['pass_end_location'].apply(lambda loc: loc[0])
final_event_pass['y'] = final_event_pass['location'].apply(lambda loc: loc[1])
final_event_pass['endy'] = final_event_pass['pass_end_location'].apply(lambda loc: loc[1])


In [18]:

final_event_pass_2['pass_end_location'] = final_event_pass_2['pass_end_location'].apply(convert_to_tuple)
final_event_pass_2['location'] = final_event_pass_2['location'].apply(convert_to_tuple)

# Extract x, y, endx, endy coordinates
final_event_pass_2['x'] = final_event_pass_2['location'].apply(lambda loc: loc[0])
final_event_pass_2['endx'] = final_event_pass_2['pass_end_location'].apply(lambda loc: loc[0])
final_event_pass_2['y'] = final_event_pass_2['location'].apply(lambda loc: loc[1])
final_event_pass_2['endy'] = final_event_pass_2['pass_end_location'].apply(lambda loc: loc[1])




In [19]:
prior_match_pass=prior_match_pass[['match_id', 'x', 'endx', 'y','endy','team']].reset_index(drop=True)

In [20]:
final_event_pass=final_event_pass[['match_id', 'x', 'endx', 'y','endy','team']].reset_index(drop=True)
final_event_pass_2=final_event_pass_2[['match_id', 'x', 'endx', 'y','endy','team']].reset_index(drop=True)

In [21]:
prior_match_pass['box_start'] = np.where((prior_match_pass.x < 20) & (prior_match_pass.y < 20), 1,
                                          np.where((prior_match_pass.x < 20) & (prior_match_pass.y >= 20) & (prior_match_pass.y < 40), 2,
                                                   np.where((prior_match_pass.x < 20) & (prior_match_pass.y >= 40) & (prior_match_pass.y < 60), 3,
                                                            np.where((prior_match_pass.x < 20) & (prior_match_pass.y >= 60), 4,
                                                                     np.where((prior_match_pass.x >= 20) & (prior_match_pass.x < 40) & (prior_match_pass.y < 20), 5,
                                                                              np.where((prior_match_pass.x >= 20) & (prior_match_pass.x < 40) & (prior_match_pass.y >= 20) & (prior_match_pass.y < 40), 6,
                                                                                       np.where((prior_match_pass.x >= 20) & (prior_match_pass.x < 40) & (prior_match_pass.y >= 40) & (prior_match_pass.y < 60), 7,
                                                                                                np.where((prior_match_pass.x >= 20) & (prior_match_pass.x < 40) & (prior_match_pass.y >= 60), 8,
                                                                                                         np.where((prior_match_pass.x >= 40) & (prior_match_pass.x < 60) & (prior_match_pass.y < 20), 9,
                                                                                                                  np.where((prior_match_pass.x >= 40) & (prior_match_pass.x < 60) & (prior_match_pass.y >= 20) & (prior_match_pass.y < 40), 10,
                                                                                                                           np.where((prior_match_pass.x >= 40) & (prior_match_pass.x < 60) & (prior_match_pass.y >= 40) & (prior_match_pass.y < 60), 11,
                                                                                                                                    np.where((prior_match_pass.x >= 40) & (prior_match_pass.x < 60) & (prior_match_pass.y >= 60), 12,
                                                                                                                                             np.where((prior_match_pass.x >= 60) & (prior_match_pass.x < 80) & (prior_match_pass.y < 20), 13,
                                                                                                                                                      np.where((prior_match_pass.x >= 60) & (prior_match_pass.x < 80) & (prior_match_pass.y >= 20) & (prior_match_pass.y < 40), 14,
                                                                                                                                                               np.where((prior_match_pass.x >= 60) & (prior_match_pass.x < 80) & (prior_match_pass.y >= 40) & (prior_match_pass.y < 60), 15,
                                                                                                                                                                        np.where((prior_match_pass.x >= 60) & (prior_match_pass.x < 80) & (prior_match_pass.y >= 60), 16,
                                                                                                                                                                                 np.where((prior_match_pass.x >= 80) & (prior_match_pass.x < 100) & (prior_match_pass.y < 20), 17,
                                                                                                                                                                                          np.where((prior_match_pass.x >= 80) & (prior_match_pass.x < 100) & (prior_match_pass.y >= 20) & (prior_match_pass.y < 40), 18,
                                                                                                                                                                                                   np.where((prior_match_pass.x >= 80) & (prior_match_pass.x < 100) & (prior_match_pass.y >= 40) & (prior_match_pass.y < 60), 19,
                                                                                                                                                                                                            np.where((prior_match_pass.x >= 80) & (prior_match_pass.x < 100) & (prior_match_pass.y >= 60), 20,
                                                                                                                                                                                                                     np.where((prior_match_pass.x >= 100) & (prior_match_pass.y < 20), 21,
                                                                                                                                                                                                                              np.where((prior_match_pass.x >= 100) & (prior_match_pass.y >= 20) & (prior_match_pass.y < 40), 22,
                                                                                                                                                                                                                                       np.where((prior_match_pass.x >= 100) & (prior_match_pass.y >= 40) & (prior_match_pass.y < 60), 23,
                                                                                                                                                                                                                                                np.where((prior_match_pass.x >= 100) & (prior_match_pass.y >= 60), 24,
                                                                                                                                                                                                                                                         0)))))))))))))))))))))))) 

# Display the DataFrame to check the result
prior_match_pass['box_end'] = np.where((prior_match_pass.endx < 20) & (prior_match_pass.endy < 20), 1,
                                          np.where((prior_match_pass.endx < 20) & (prior_match_pass.endy >= 20) & (prior_match_pass.endy < 40), 2,
                                                   np.where((prior_match_pass.endx < 20) & (prior_match_pass.endy >= 40) & (prior_match_pass.endy < 60), 3,
                                                            np.where((prior_match_pass.endx < 20) & (prior_match_pass.endy >= 60), 4,
                                                                     np.where((prior_match_pass.endx >= 20) & (prior_match_pass.endx < 40) & (prior_match_pass.endy < 20), 5,
                                                                              np.where((prior_match_pass.endx >= 20) & (prior_match_pass.endx < 40) & (prior_match_pass.endy >= 20) & (prior_match_pass.endy < 40), 6,
                                                                                       np.where((prior_match_pass.endx >= 20) & (prior_match_pass.endx < 40) & (prior_match_pass.endy >= 40) & (prior_match_pass.endy < 60), 7,
                                                                                                np.where((prior_match_pass.endx >= 20) & (prior_match_pass.endx < 40) & (prior_match_pass.endy >= 60), 8,
                                                                                                         np.where((prior_match_pass.endx >= 40) & (prior_match_pass.endx < 60) & (prior_match_pass.endy < 20), 9,
                                                                                                                  np.where((prior_match_pass.endx >= 40) & (prior_match_pass.endx < 60) & (prior_match_pass.endy >= 20) & (prior_match_pass.endy < 40), 10,
                                                                                                                           np.where((prior_match_pass.endx >= 40) & (prior_match_pass.endx < 60) & (prior_match_pass.endy >= 40) & (prior_match_pass.endy < 60), 11,
                                                                                                                                    np.where((prior_match_pass.endx >= 40) & (prior_match_pass.endx < 60) & (prior_match_pass.endy >= 60), 12,
                                                                                                                                             np.where((prior_match_pass.endx >= 60) & (prior_match_pass.endx < 80) & (prior_match_pass.endy < 20), 13,
                                                                                                                                                      np.where((prior_match_pass.endx >= 60) & (prior_match_pass.endx < 80) & (prior_match_pass.endy >= 20) & (prior_match_pass.endy < 40), 14,
                                                                                                                                                               np.where((prior_match_pass.endx >= 60) & (prior_match_pass.endx < 80) & (prior_match_pass.endy >= 40) & (prior_match_pass.endy < 60), 15,
                                                                                                                                                                        np.where((prior_match_pass.endx >= 60) & (prior_match_pass.endx < 80) & (prior_match_pass.endy >= 60), 16,
                                                                                                                                                                                 np.where((prior_match_pass.endx >= 80) & (prior_match_pass.endx < 100) & (prior_match_pass.endy < 20), 17,
                                                                                                                                                                                          np.where((prior_match_pass.endx >= 80) & (prior_match_pass.endx < 100) & (prior_match_pass.endy >= 20) & (prior_match_pass.endy < 40), 18,
                                                                                                                                                                                                   np.where((prior_match_pass.endx >= 80) & (prior_match_pass.endx < 100) & (prior_match_pass.endy >= 40) & (prior_match_pass.endy < 60), 19,
                                                                                                                                                                                                            np.where((prior_match_pass.endx >= 80) & (prior_match_pass.endx < 100) & (prior_match_pass.endy >= 60), 20,
                                                                                                                                                                                                                     np.where((prior_match_pass.endx >= 100) & (prior_match_pass.endy < 20), 21,
                                                                                                                                                                                                                              np.where((prior_match_pass.endx >= 100) & (prior_match_pass.endy >= 20) & (prior_match_pass.endy < 40), 22,
                                                                                                                                                                                                                                       np.where((prior_match_pass.endx >= 100) & (prior_match_pass.endy >= 40) & (prior_match_pass.endy < 60), 23,
                                                                                                                                                                                                                                                np.where((prior_match_pass.endx >= 100) & (prior_match_pass.endy >= 60), 24,
                                                                                                                                                                                                                                                         0)))))))))))))))))))))))) 

# Display the DataFrame to check the result

In [22]:
final_event_pass['box_start'] = np.where((final_event_pass.x < 20) & (final_event_pass.y < 20), 1,
                                          np.where((final_event_pass.x < 20) & (final_event_pass.y >= 20) & (final_event_pass.y < 40), 2,
                                                   np.where((final_event_pass.x < 20) & (final_event_pass.y >= 40) & (final_event_pass.y < 60), 3,
                                                            np.where((final_event_pass.x < 20) & (final_event_pass.y >= 60), 4,
                                                                     np.where((final_event_pass.x >= 20) & (final_event_pass.x < 40) & (final_event_pass.y < 20), 5,
                                                                              np.where((final_event_pass.x >= 20) & (final_event_pass.x < 40) & (final_event_pass.y >= 20) & (final_event_pass.y < 40), 6,
                                                                                       np.where((final_event_pass.x >= 20) & (final_event_pass.x < 40) & (final_event_pass.y >= 40) & (final_event_pass.y < 60), 7,
                                                                                                np.where((final_event_pass.x >= 20) & (final_event_pass.x < 40) & (final_event_pass.y >= 60), 8,
                                                                                                         np.where((final_event_pass.x >= 40) & (final_event_pass.x < 60) & (final_event_pass.y < 20), 9,
                                                                                                                  np.where((final_event_pass.x >= 40) & (final_event_pass.x < 60) & (final_event_pass.y >= 20) & (final_event_pass.y < 40), 10,
                                                                                                                           np.where((final_event_pass.x >= 40) & (final_event_pass.x < 60) & (final_event_pass.y >= 40) & (final_event_pass.y < 60), 11,
                                                                                                                                    np.where((final_event_pass.x >= 40) & (final_event_pass.x < 60) & (final_event_pass.y >= 60), 12,
                                                                                                                                             np.where((final_event_pass.x >= 60) & (final_event_pass.x < 80) & (final_event_pass.y < 20), 13,
                                                                                                                                                      np.where((final_event_pass.x >= 60) & (final_event_pass.x < 80) & (final_event_pass.y >= 20) & (final_event_pass.y < 40), 14,
                                                                                                                                                               np.where((final_event_pass.x >= 60) & (final_event_pass.x < 80) & (final_event_pass.y >= 40) & (final_event_pass.y < 60), 15,
                                                                                                                                                                        np.where((final_event_pass.x >= 60) & (final_event_pass.x < 80) & (final_event_pass.y >= 60), 16,
                                                                                                                                                                                 np.where((final_event_pass.x >= 80) & (final_event_pass.x < 100) & (final_event_pass.y < 20), 17,
                                                                                                                                                                                          np.where((final_event_pass.x >= 80) & (final_event_pass.x < 100) & (final_event_pass.y >= 20) & (final_event_pass.y < 40), 18,
                                                                                                                                                                                                   np.where((final_event_pass.x >= 80) & (final_event_pass.x < 100) & (final_event_pass.y >= 40) & (final_event_pass.y < 60), 19,
                                                                                                                                                                                                            np.where((final_event_pass.x >= 80) & (final_event_pass.x < 100) & (final_event_pass.y >= 60), 20,
                                                                                                                                                                                                                     np.where((final_event_pass.x >= 100) & (final_event_pass.y < 20), 21,
                                                                                                                                                                                                                              np.where((final_event_pass.x >= 100) & (final_event_pass.y >= 20) & (final_event_pass.y < 40), 22,
                                                                                                                                                                                                                                       np.where((final_event_pass.x >= 100) & (final_event_pass.y >= 40) & (final_event_pass.y < 60), 23,
                                                                                                                                                                                                                                                np.where((final_event_pass.x >= 100) & (final_event_pass.y >= 60), 24,
                                                                                                                                                                                                                                                         0)))))))))))))))))))))))) 

# Display the DataFrame to check the result
final_event_pass['box_end'] = np.where((final_event_pass.endx < 20) & (final_event_pass.endy < 20), 1,
                                          np.where((final_event_pass.endx < 20) & (final_event_pass.endy >= 20) & (final_event_pass.endy < 40), 2,
                                                   np.where((final_event_pass.endx < 20) & (final_event_pass.endy >= 40) & (final_event_pass.endy < 60), 3,
                                                            np.where((final_event_pass.endx < 20) & (final_event_pass.endy >= 60), 4,
                                                                     np.where((final_event_pass.endx >= 20) & (final_event_pass.endx < 40) & (final_event_pass.endy < 20), 5,
                                                                              np.where((final_event_pass.endx >= 20) & (final_event_pass.endx < 40) & (final_event_pass.endy >= 20) & (final_event_pass.endy < 40), 6,
                                                                                       np.where((final_event_pass.endx >= 20) & (final_event_pass.endx < 40) & (final_event_pass.endy >= 40) & (final_event_pass.endy < 60), 7,
                                                                                                np.where((final_event_pass.endx >= 20) & (final_event_pass.endx < 40) & (final_event_pass.endy >= 60), 8,
                                                                                                         np.where((final_event_pass.endx >= 40) & (final_event_pass.endx < 60) & (final_event_pass.endy < 20), 9,
                                                                                                                  np.where((final_event_pass.endx >= 40) & (final_event_pass.endx < 60) & (final_event_pass.endy >= 20) & (final_event_pass.endy < 40), 10,
                                                                                                                           np.where((final_event_pass.endx >= 40) & (final_event_pass.endx < 60) & (final_event_pass.endy >= 40) & (final_event_pass.endy < 60), 11,
                                                                                                                                    np.where((final_event_pass.endx >= 40) & (final_event_pass.endx < 60) & (final_event_pass.endy >= 60), 12,
                                                                                                                                             np.where((final_event_pass.endx >= 60) & (final_event_pass.endx < 80) & (final_event_pass.endy < 20), 13,
                                                                                                                                                      np.where((final_event_pass.endx >= 60) & (final_event_pass.endx < 80) & (final_event_pass.endy >= 20) & (final_event_pass.endy < 40), 14,
                                                                                                                                                               np.where((final_event_pass.endx >= 60) & (final_event_pass.endx < 80) & (final_event_pass.endy >= 40) & (final_event_pass.endy < 60), 15,
                                                                                                                                                                        np.where((final_event_pass.endx >= 60) & (final_event_pass.endx < 80) & (final_event_pass.endy >= 60), 16,
                                                                                                                                                                                 np.where((final_event_pass.endx >= 80) & (final_event_pass.endx < 100) & (final_event_pass.endy < 20), 17,
                                                                                                                                                                                          np.where((final_event_pass.endx >= 80) & (final_event_pass.endx < 100) & (final_event_pass.endy >= 20) & (final_event_pass.endy < 40), 18,
                                                                                                                                                                                                   np.where((final_event_pass.endx >= 80) & (final_event_pass.endx < 100) & (final_event_pass.endy >= 40) & (final_event_pass.endy < 60), 19,
                                                                                                                                                                                                            np.where((final_event_pass.endx >= 80) & (final_event_pass.endx < 100) & (final_event_pass.endy >= 60), 20,
                                                                                                                                                                                                                     np.where((final_event_pass.endx >= 100) & (final_event_pass.endy < 20), 21,
                                                                                                                                                                                                                              np.where((final_event_pass.endx >= 100) & (final_event_pass.endy >= 20) & (final_event_pass.endy < 40), 22,
                                                                                                                                                                                                                                       np.where((final_event_pass.endx >= 100) & (final_event_pass.endy >= 40) & (final_event_pass.endy < 60), 23,
                                                                                                                                                                                                                                                np.where((final_event_pass.endx >= 100) & (final_event_pass.endy >= 60), 24,
                                                                                                                                                                                                                                                         0)))))))))))))))))))))))) 

# Display the DataFrame to check the result

In [23]:
final_event_pass_2['box_start'] = np.where((final_event_pass_2.x < 20) & (final_event_pass_2.y < 20), 1,
                                          np.where((final_event_pass_2.x < 20) & (final_event_pass_2.y >= 20) & (final_event_pass_2.y < 40), 2,
                                                   np.where((final_event_pass_2.x < 20) & (final_event_pass_2.y >= 40) & (final_event_pass_2.y < 60), 3,
                                                            np.where((final_event_pass_2.x < 20) & (final_event_pass_2.y >= 60), 4,
                                                                     np.where((final_event_pass_2.x >= 20) & (final_event_pass_2.x < 40) & (final_event_pass_2.y < 20), 5,
                                                                              np.where((final_event_pass_2.x >= 20) & (final_event_pass_2.x < 40) & (final_event_pass_2.y >= 20) & (final_event_pass_2.y < 40), 6,
                                                                                       np.where((final_event_pass_2.x >= 20) & (final_event_pass_2.x < 40) & (final_event_pass_2.y >= 40) & (final_event_pass_2.y < 60), 7,
                                                                                                np.where((final_event_pass_2.x >= 20) & (final_event_pass_2.x < 40) & (final_event_pass_2.y >= 60), 8,
                                                                                                         np.where((final_event_pass_2.x >= 40) & (final_event_pass_2.x < 60) & (final_event_pass_2.y < 20), 9,
                                                                                                                  np.where((final_event_pass_2.x >= 40) & (final_event_pass_2.x < 60) & (final_event_pass_2.y >= 20) & (final_event_pass_2.y < 40), 10,
                                                                                                                           np.where((final_event_pass_2.x >= 40) & (final_event_pass_2.x < 60) & (final_event_pass_2.y >= 40) & (final_event_pass_2.y < 60), 11,
                                                                                                                                    np.where((final_event_pass_2.x >= 40) & (final_event_pass_2.x < 60) & (final_event_pass_2.y >= 60), 12,
                                                                                                                                             np.where((final_event_pass_2.x >= 60) & (final_event_pass_2.x < 80) & (final_event_pass_2.y < 20), 13,
                                                                                                                                                      np.where((final_event_pass_2.x >= 60) & (final_event_pass_2.x < 80) & (final_event_pass_2.y >= 20) & (final_event_pass_2.y < 40), 14,
                                                                                                                                                               np.where((final_event_pass_2.x >= 60) & (final_event_pass_2.x < 80) & (final_event_pass_2.y >= 40) & (final_event_pass_2.y < 60), 15,
                                                                                                                                                                        np.where((final_event_pass_2.x >= 60) & (final_event_pass_2.x < 80) & (final_event_pass_2.y >= 60), 16,
                                                                                                                                                                                 np.where((final_event_pass_2.x >= 80) & (final_event_pass_2.x < 100) & (final_event_pass_2.y < 20), 17,
                                                                                                                                                                                          np.where((final_event_pass_2.x >= 80) & (final_event_pass_2.x < 100) & (final_event_pass_2.y >= 20) & (final_event_pass_2.y < 40), 18,
                                                                                                                                                                                                   np.where((final_event_pass_2.x >= 80) & (final_event_pass_2.x < 100) & (final_event_pass_2.y >= 40) & (final_event_pass_2.y < 60), 19,
                                                                                                                                                                                                            np.where((final_event_pass_2.x >= 80) & (final_event_pass_2.x < 100) & (final_event_pass_2.y >= 60), 20,
                                                                                                                                                                                                                     np.where((final_event_pass_2.x >= 100) & (final_event_pass_2.y < 20), 21,
                                                                                                                                                                                                                              np.where((final_event_pass_2.x >= 100) & (final_event_pass_2.y >= 20) & (final_event_pass_2.y < 40), 22,
                                                                                                                                                                                                                                       np.where((final_event_pass_2.x >= 100) & (final_event_pass_2.y >= 40) & (final_event_pass_2.y < 60), 23,
                                                                                                                                                                                                                                                np.where((final_event_pass_2.x >= 100) & (final_event_pass_2.y >= 60), 24,
                                                                                                                                                                                                                                                         0)))))))))))))))))))))))) 

# Display the DataFrame to check the result
final_event_pass_2['box_end'] = np.where((final_event_pass_2.endx < 20) & (final_event_pass_2.endy < 20), 1,
                                          np.where((final_event_pass_2.endx < 20) & (final_event_pass_2.endy >= 20) & (final_event_pass_2.endy < 40), 2,
                                                   np.where((final_event_pass_2.endx < 20) & (final_event_pass_2.endy >= 40) & (final_event_pass_2.endy < 60), 3,
                                                            np.where((final_event_pass_2.endx < 20) & (final_event_pass_2.endy >= 60), 4,
                                                                     np.where((final_event_pass_2.endx >= 20) & (final_event_pass_2.endx < 40) & (final_event_pass_2.endy < 20), 5,
                                                                              np.where((final_event_pass_2.endx >= 20) & (final_event_pass_2.endx < 40) & (final_event_pass_2.endy >= 20) & (final_event_pass_2.endy < 40), 6,
                                                                                       np.where((final_event_pass_2.endx >= 20) & (final_event_pass_2.endx < 40) & (final_event_pass_2.endy >= 40) & (final_event_pass_2.endy < 60), 7,
                                                                                                np.where((final_event_pass_2.endx >= 20) & (final_event_pass_2.endx < 40) & (final_event_pass_2.endy >= 60), 8,
                                                                                                         np.where((final_event_pass_2.endx >= 40) & (final_event_pass_2.endx < 60) & (final_event_pass_2.endy < 20), 9,
                                                                                                                  np.where((final_event_pass_2.endx >= 40) & (final_event_pass_2.endx < 60) & (final_event_pass_2.endy >= 20) & (final_event_pass_2.endy < 40), 10,
                                                                                                                           np.where((final_event_pass_2.endx >= 40) & (final_event_pass_2.endx < 60) & (final_event_pass_2.endy >= 40) & (final_event_pass_2.endy < 60), 11,
                                                                                                                                    np.where((final_event_pass_2.endx >= 40) & (final_event_pass_2.endx < 60) & (final_event_pass_2.endy >= 60), 12,
                                                                                                                                             np.where((final_event_pass_2.endx >= 60) & (final_event_pass_2.endx < 80) & (final_event_pass_2.endy < 20), 13,
                                                                                                                                                      np.where((final_event_pass_2.endx >= 60) & (final_event_pass_2.endx < 80) & (final_event_pass_2.endy >= 20) & (final_event_pass_2.endy < 40), 14,
                                                                                                                                                               np.where((final_event_pass_2.endx >= 60) & (final_event_pass_2.endx < 80) & (final_event_pass_2.endy >= 40) & (final_event_pass_2.endy < 60), 15,
                                                                                                                                                                        np.where((final_event_pass_2.endx >= 60) & (final_event_pass_2.endx < 80) & (final_event_pass_2.endy >= 60), 16,
                                                                                                                                                                                 np.where((final_event_pass_2.endx >= 80) & (final_event_pass_2.endx < 100) & (final_event_pass_2.endy < 20), 17,
                                                                                                                                                                                          np.where((final_event_pass_2.endx >= 80) & (final_event_pass_2.endx < 100) & (final_event_pass_2.endy >= 20) & (final_event_pass_2.endy < 40), 18,
                                                                                                                                                                                                   np.where((final_event_pass_2.endx >= 80) & (final_event_pass_2.endx < 100) & (final_event_pass_2.endy >= 40) & (final_event_pass_2.endy < 60), 19,
                                                                                                                                                                                                            np.where((final_event_pass_2.endx >= 80) & (final_event_pass_2.endx < 100) & (final_event_pass_2.endy >= 60), 20,
                                                                                                                                                                                                                     np.where((final_event_pass_2.endx >= 100) & (final_event_pass_2.endy < 20), 21,
                                                                                                                                                                                                                              np.where((final_event_pass_2.endx >= 100) & (final_event_pass_2.endy >= 20) & (final_event_pass_2.endy < 40), 22,
                                                                                                                                                                                                                                       np.where((final_event_pass_2.endx >= 100) & (final_event_pass_2.endy >= 40) & (final_event_pass_2.endy < 60), 23,
                                                                                                                                                                                                                                                np.where((final_event_pass_2.endx >= 100) & (final_event_pass_2.endy >= 60), 24,
                                                                                                                                                                                                                                                         0)))))))))))))))))))))))) 

# Display the DataFrame to check the result

In [24]:
transition_counts = prior_match_pass.groupby(['box_start', 'box_end']).size().reset_index(name='count')

In [25]:
final_counts = final_event_pass.groupby(['box_start', 'box_end']).size().reset_index(name='count')

In [26]:
final_counts_2 = final_event_pass_2.groupby(['box_start', 'box_end']).size().reset_index(name='count')

In [27]:
grouped_by_start_final_2 = final_counts_2.groupby('box_start')['count'].transform('sum')
final_counts_2['probability'] = final_counts_2['count'] / grouped_by_start_final_2

In [28]:
grouped_by_start_final = final_counts.groupby('box_start')['count'].transform('sum')
final_counts['probability'] = final_counts['count'] / grouped_by_start_final

In [29]:
grouped_by_start = transition_counts.groupby('box_start')['count'].transform('sum')
transition_counts['probability'] = transition_counts['count'] / grouped_by_start

In [30]:
# Create the transition matrix as a DataFrame
transition_matrix = transition_counts.pivot(index='box_start', columns='box_end', values='probability')

# Fill NaN values with 0 (if there's no transition between a pair of boxes)
transition_matrix = transition_matrix.fillna(0)

# Display the transition matrix
transition_matrix

box_end,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
box_start,,,,,,,,,,,,,,,,,,,,,
1,0.133333,0.400000,0.033333,0.000000,0.300000,0.066667,0.000000,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.052632,0.142857,0.120301,0.007519,0.067669,0.135338,0.157895,0.030075,0.030075,0.030075,...,0.022556,0.007519,0.030075,0.000000,0.007519,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.029630,0.111111,0.170370,0.022222,0.059259,0.096296,0.118519,0.059259,0.022222,0.059259,...,0.022222,0.066667,0.007407,0.000000,0.007407,0.007407,0.000000,0.000000,0.007407,0.000000
4,0.000000,0.000000,0.115385,0.269231,0.038462,0.000000,0.038462,0.423077,0.000000,0.000000,...,0.000000,0.038462,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000
5,0.056911,0.146341,0.000000,0.000000,0.292683,0.154472,0.024390,0.000000,0.227642,0.056911,...,0.000000,0.008130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.023077,0.115385,0.084615,0.000000,0.107692,0.184615,0.100000,0.038462,0.107692,0.092308,...,0.015385,0.000000,0.007692,0.000000,0.007692,0.000000,0.000000,0.000000,0.007692,0.007692
7,0.000000,0.056911,0.065041,0.008130,0.056911,0.130081,0.186992,0.097561,0.024390,0.040650,...,0.000000,0.008130,0.000000,0.000000,0.000000,0.008130,0.000000,0.008130,0.008130,0.000000
8,0.000000,0.023256,0.151163,0.069767,0.000000,0.046512,0.069767,0.267442,0.000000,0.000000,...,0.000000,0.011628,0.000000,0.000000,0.000000,0.011628,0.000000,0.000000,0.000000,0.000000
9,0.012048,0.036145,0.000000,0.000000,0.096386,0.048193,0.012048,0.000000,0.289157,0.150602,...,0.000000,0.000000,0.024096,0.000000,0.000000,0.012048,0.012048,0.012048,0.000000,0.012048


In [31]:
# Create the transition matrix as a DataFrame
final_matrix = final_counts.pivot(index='box_start', columns='box_end', values='probability')

# Fill Nan values with 0 (if there's no transition between a pair of boxes)
final_matrix = final_matrix.fillna(0)

# Display the transition matrix
final_matrix

box_end,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
box_start,,,,,,,,,,,,,,,,,,,,,
2,0.166667,0.666667,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.200000,0.000000,0.133333,0.000000,0.333333,0.200000,0.066667,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.250000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.076923,0.307692,0.230769,0.076923,0.153846,0.076923,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000
7,0.111111,0.222222,0.000000,0.000000,0.222222,0.000000,0.222222,0.111111,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.250000,0.062500,0.000000,0.062500,0.125000,0.250000,0.000000,0.000000,0.000000,...,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.307692,0.384615,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,0.000000,0.100000,0.050000,0.000000,0.150000,0.300000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
merged = transition_counts.merge(final_counts, on=['box_start', 'box_end'], how='outer', suffixes=('_prior', '_final'))
merged['count_prior'] = merged['count_prior'].fillna(0)
merged['count_final'] = merged['count_final'].fillna(0)
merged['probability_prior'] = merged['probability_prior'].fillna(0)
merged['probability_final'] = merged['probability_final'].fillna(0)
merged['posterior_count'] = merged['count_prior'] + merged['count_final']
merged['group_sum'] = merged.groupby('box_start')['posterior_count'].transform('sum')
merged['posterior_probability'] = merged['posterior_count'] / merged['group_sum']
merged['posterior_probability'] = merged['posterior_probability'].fillna(0)
merged=merged.merge(final_counts_2, on=['box_start', 'box_end'], how='outer')
merged.rename(columns={'count':'count_2','probability':'probability_2'},inplace=True)
merged['count_2'] = merged['count_2'].fillna(0)
merged['probability_2'] = merged['probability_2'].fillna(0)
merged

,box_start,box_end,count_prior,probability_prior,count_final,probability_final,posterior_count,group_sum,posterior_probability,count_2,probability_2
0,1,1,4.0,0.133333,0.0,0.0,4.0,30.0,0.133333,0.0,0.000000
1,1,2,12.0,0.400000,0.0,0.0,12.0,30.0,0.400000,0.0,0.000000
2,1,3,1.0,0.033333,0.0,0.0,1.0,30.0,0.033333,0.0,0.000000
3,1,5,9.0,0.300000,0.0,0.0,9.0,30.0,0.300000,0.0,0.000000
4,1,6,2.0,0.066667,0.0,0.0,2.0,30.0,0.066667,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
312,24,20,17.0,0.209877,0.0,0.0,17.0,86.0,0.197674,3.0,0.333333
313,24,21,2.0,0.024691,0.0,0.0,2.0,86.0,0.023256,0.0,0.000000
314,24,22,9.0,0.111111,2.0,0.4,11.0,86.0,0.127907,3.0,0.333333
315,24,23,24.0,0.296296,2.0,0.4,26.0,86.0,0.302326,1.0,0.111111


In [33]:
merged_map=merged[['box_start','box_end','probability_prior','probability_final','posterior_probability','probability_2']]
merged_map.rename(columns={'probability_final':'probability_1'},inplace=True)
merged_map

/var/folders/w1/v_96k9nn4k73369wp7ddnydw0000gn/T/ipykernel_23697/3908578259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_map.rename(columns={'probability_final':'probability_1'},inplace=True)


,box_start,box_end,probability_prior,probability_1,posterior_probability,probability_2
0,1,1,0.133333,0.0,0.133333,0.000000
1,1,2,0.400000,0.0,0.400000,0.000000
2,1,3,0.033333,0.0,0.033333,0.000000
3,1,5,0.300000,0.0,0.300000,0.000000
4,1,6,0.066667,0.0,0.066667,0.000000
...,...,...,...,...,...,...
312,24,20,0.209877,0.0,0.197674,0.333333
313,24,21,0.024691,0.0,0.023256,0.000000
314,24,22,0.111111,0.4,0.127907,0.333333
315,24,23,0.296296,0.4,0.302326,0.111111


In [34]:
# Create posterior transition matrix (box_start → box_end)
posterior_matrix = merged.pivot(index='box_start', columns='box_end', values='posterior_probability')

# Replace NaNs with 0 (in case some transitions never happened)
posterior_matrix = posterior_matrix.fillna(0)



In [35]:
#merged_map.to_csv('/Users/preciousprince/Downloads/merged_map.csv')